In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject\\research'

In [3]:
os.chdir("../")

In [24]:
%pwd

'c:\\Users\\shalm\\source\\repos\\EndToEndMachineLearningProject'

In [36]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict   # Yaml file always return dictionary format , so datatype is Dict

In [26]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:  # return type : DataValidationConfig
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [47]:
import os
from mlProject import logger
import pandas as pd

In [48]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    
    def validate_all_columns(self)-> bool:  #we are only checking if the columns are present. We can also write a method for checking the datatype of the column as well.
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:    # Open the text file where you want to store
                        f.write(f"Validation status: {validation_status}")  # write in that file
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

 We checked if columns are available , for datatype check you can write a method. This is another layer of validation. (to avoid errors)

In [49]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-03-19 10:43:09,625: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-19 10:43:09,631: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 10:43:09,645: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-19 10:43:09,647: INFO: common: created directory at: artifacts]
[2024-03-19 10:43:09,650: INFO: common: created directory at: artifacts/data_validation]
